In [1]:
from miditok import REMI, TokenizerConfig
from miditok.pytorch_data import DatasetMIDI, DataCollator
from miditok.utils import split_files_for_training
from torch.utils.data import DataLoader
from pathlib import Path

# Creating a multitrack tokenizer, read the doc to explore all the parameters
config = TokenizerConfig(num_velocities=16, use_chords=True, use_programs=True)
tokenizer = REMI(config)



C:\Users\keisu\AppData\Roaming\Python\Python39\site-packages\miditok\tokenizations\remi.py:77: UserWarning: Attribute controls are not compatible with 'config.one_token_stream_for_programs' and multi-vocabulary tokenizers. Disabling them from the config.
  super().__init__(tokenizer_config, params)


In [2]:
from LSTMwithAtt import LSTMwithAtt
import torch

from torch.nn.utils.rnn import pad_sequence
import time

import icecream as ic


import os

from symusic import Score

import datetime

from tqdm import tqdm  # 進行状況を可視化するライブラリ

from transformers import GPT2Config, GPT2LMHeadModel

In [3]:
#device setting

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
# Train the tokenizer with Byte Pair Encoding (BPE)
#files_paths = list(Path("../datasets").glob("**/*.mid"))
# dataset_dir = Path("/Users/hapticslab/Programming/humusic/datasets/orchestra")
dataset_dir = Path("C:/Users/keisu/Programming/humor/humusic/datasets/orchestra")
dataset_dir = Path("C:/Users/keisu/Programming/humor/humusic/datasets/piano1_only")
files_paths = list(dataset_dir.glob("**/*.mid"))

tokenizer.train(vocab_size=30000, files_paths=files_paths)

#tokenizer.save(Path("path", "to", "save", "tokenizer.json"))
# tokenizer.save(Path("/Users/hapticslab/Programming/humusic/midi_generator/tokens/token_folder_name/tokenizer.json"))
#tokenizer.save(Path("C:/Users/keisu/Programming/humor/humusic/midi_generator/tokens/orchestra_token/tokenizer.json"))
tokenizer.save(Path("C:/Users/keisu/Programming/humor/humusic/midi_generator/tokens/piano1_token/tokenizer.json"))
# And pushing it to the Hugging Face hub (you can download it back with .from_pretrained)
#tokenizer.push_to_hub("username/model-name", private=True, token="your_hf_token")

# Split MIDIs into smaller chunks for training
# dataset_chunks_dir = Path("/Users/hapticslab/Programming/humusic/midi_generator/midi_chunk")
#dataset_chunks_dir = Path("C:/Users/keisu/Programming/humor/humusic/midi_generator/midi_chunk/orchestra")
dataset_chunks_dir = Path("C:/Users/keisu/Programming/humor/humusic/midi_generator/midi_chunk/piano1_only")

'''
split_files_for_training(
    files_paths=files_paths,
    tokenizer=tokenizer,
    save_dir=dataset_chunks_dir,
    max_seq_len=1024,
)
'''
# Create a Dataset, a DataLoader and a collator to train a model
dataset = DatasetMIDI(
    files_paths=list(dataset_chunks_dir.glob("**/*.mid")),
    tokenizer=tokenizer,
    max_seq_len=1024,
)
collator = DataCollator(tokenizer.pad_token_id, copy_inputs_as_labels=True)
dataloader = DataLoader(dataset, batch_size=16, collate_fn=collator)

In [6]:
# LSMT with Attention
'''
hidden_size = 200
model = LSTMwithAtt(4, hidden_size)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss(ignore_index=-1)

num_epoch = 10

model.train()

start_time = time.time()

for epoch in range(num_epoch):
    epoch_start_time = time.time()
    i = 0

    for batch in dataloader:
        xs, ys ,ans = [],[],[]

        for k in range(len(batch)):
            
            input = batch["input_ids"].to(device)
            
            tid = batch[k]
            xs.append(torch.LongTensor(tid[1:]))
            ys.append(torch.LongTensor(tid[:-1]))
            ans.append(torch.LongTensor(tid[1:]))
        
        encoder_input = pad_sequence(xs, batch_first=True).to(device)
        decoder_input = pad_sequence(ys, batch_first=True).to(device)
        answer = pad_sequence(ans, batch_first=True, padding_value=-1.0).to(device)

        out = model(encoder_input, decoder_input)
        loss = criterion(out[0],answer[0])
        for h in range(1,len(out)):
            loss += criterion(out[h],answer[h])
        print(f'| epoch {epoch:3d} | {i:5d}/{len(dataloader)} batches | loss {loss.item():5.2f} | time {time.time()-epoch_start_time:5.2f}s')
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        i += 1
    
    print(f'epoch {epoch+1}/{num_epoch} time : {time.time()-epoch_start_time}')
    outfile = "models/" + "test"+ str(epoch+1) + ".model"
    torch.save(model.state_dict(),outfile)'''

None


In [12]:



config = GPT2Config(
    vocab_size=tokenizer.vocab_size,
    n_positions=2048,
    n_ctx=2048,
    n_embd=256,
    n_layer=8,
    n_head=8,
    
    #bos_token_id=tokenizer["BOS_None"],
    #eos_token_id=tokenizer["EOS_None"],
    pad_token_id=tokenizer.pad_token_id
    
    )

model = GPT2LMHeadModel(config).to(device)

# 損失関数とオプティマイザ
criterion = torch.nn.CrossEntropyLoss()  # トークンの予測タスクに使う
optimizer = torch.optim.AdamW(model.parameters(), lr=0.01)  # 適切な学習率を設定

In [13]:


# 学習ループ
epochs = 2 # エポック数
for epoch in range(epochs):
    model.train()  # モデルを学習モードに切り替え
    total_loss = 0

    for batch in tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}"):
        inputs = batch["input_ids"].to(device)  # トークンID列
        labels = batch["labels"].to(device)    # 正解ラベル
        attention_mask = batch["attention_mask"].to(device)  # マスク（任意）

        # 勾配を初期化
        optimizer.zero_grad()

        # モデルの前方計算
        outputs = model(
            input_ids=inputs,
            labels=labels,
            attention_mask=attention_mask,
        )
        loss = outputs.loss  # GPT2LMHeadModelは自動で損失を計算する

        # 勾配の計算とパラメータの更新
        loss.backward()
        optimizer.step()

        # 損失を記録
        total_loss += loss.item()

    # エポックごとの平均損失を出力
    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f}")
    
    model.save_pretrained("models/piano1_only/test" + str(epoch+1) + ".pt")

Epoch 1/2: 100%|██████████| 29/29 [02:33<00:00,  5.30s/it]


Epoch 1/2 - Loss: 10.4803


Epoch 2/2: 100%|██████████| 29/29 [02:26<00:00,  5.04s/it]

Epoch 2/2 - Loss: 9.8964


### promptの選択

In [14]:


# 入力トークン列の例
prompt = ("prompt.mid")

input_ids = torch.tensor([tokenizer(prompt)]).to(device)
input_ids

tensor([[  445,   767,   234, 10777,  2616,  2793,  2314,  2008,  1360,  1855,
          1261,  2149,  1538,  4742,    58,  3483,  1005]], device='cuda:0')

### モデルの選択

In [15]:
#regenerate

# モデルの読み込み

model = GPT2LMHeadModel.from_pretrained("models/piano1_only/test2.pt")
model.to(device)
model.eval()

# モデルによる生成
# output = model.generate(
#     input_ids=input_ids,
#     max_length=1024,
#     num_beams=5,
#     no_repeat_ngram_size=2,
# )

# モデルによる生成
output = model.generate(
    input_ids=input_ids,
    max_length=1024,
    num_beams=5,  # ビームサーチ
    no_repeat_ngram_size=2,  # 同じn-gramを繰り返さない
    temperature=2.0,  # サンプリングの多様性を調整
    top_k=200,  # 上位k個のトークンをサンプリング
    top_p=0.90,  # 累積確率cutoff
    
)

C:\Users\keisu\AppData\Roaming\Python\Python39\site-packages\transformers\generation\configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `2.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
C:\Users\keisu\AppData\Roaming\Python\Python39\site-packages\transformers\generation\configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
C:\Users\keisu\AppData\Roaming\Python\Python39\site-packages\transformers\generation\configuration_utils.py:537: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `200` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


In [16]:
# トークン列をデコードして結果を表示
generated = tokenizer.decode(output[0].tolist())
print("Generated MIDI Tokens:", generated)

generated.dump_midi("generated_test5.mid")

Generated MIDI Tokens: Score(ttype=Tick, tpq=8, begin=0, end=5480, tracks=1, notes=47, time_sig=1, key_sig=0, markers=0)


In [17]:
del model, dataset, dataloader, collator
torch.cuda.empty_cache()

In [18]:
#pygameによる再生

import time
import pygame


'''
# Play the MIDI file in the current directory
for msg in mid.play():
    time.sleep(msg.time)
    if not msg.is_meta:
        print(msg)
'''
# Initialize pygame
pygame.init()

# Set up the mixer
pygame.mixer.init()

# Load the MIDI file
#pygame.mixer.music.load('bach_850.mid')
pygame.mixer.music.load("generated_test5.mid")
#pygame.mixer.music.load('test.mid')

# Play the MIDI file
pygame.mixer.music.set_volume(0.25)
pygame.mixer.music.play()


# Keep the program running until the music stops
while pygame.mixer.music.get_busy():
    
    pygame.time.Clock().tick(10)

# Quit pygame
pygame.quit()

# test.midの作成セル

In [31]:
# まずは1小節のnote（ド、C4）を1つ生成してみる
import mido
from mido import Message, MidiFile, MidiTrack, MetaMessage

mid = MidiFile()
track = MidiTrack()
mid.tracks.append(track)
track.append(MetaMessage('set_tempo', tempo=mido.bpm2tempo(120))) # bpm120


track.append(Message('note_on', note=60, velocity=100, time=0))
track.append(Message('note_off', note=60, velocity=100, time=480))
track.append(Message('note_on', note=74, velocity=57, time=120))
track.append(Message('note_on', note=76, velocity=57, time=120))
track.append(Message('note_off', note=74, velocity=0, time=20))
track.append(Message('note_on', note=78, velocity=57, time=100))
track.append(Message('note_off', note=76, velocity=0, time=20))
track.append(Message('note_on', note=69, velocity=60, time=100))
track.append(Message('note_off', note=78, velocity=0, time=20))
track.append(Message('note_off', note=69, velocity=0, time=100))
track.append(Message('note_on', note=78, velocity=57, time=0))
track.append(Message('note_on', note=76, velocity=57, time=120))
track.append(Message('note_off', note=78, velocity=0, time=20))
track.append(Message('note_on', note=74, velocity=57, time=100))
track.append(Message('note_off', note=76, velocity=0, time=20))
track.append(Message('note_on', note=81, velocity=62, time=100))
track.append(Message('note_off', note=74, velocity=0, time=20))
track.append(Message('note_off', note=81, velocity=0, time=100))
track.append(Message('note_on', note=74, velocity=58, time=0))
track.append(Message('note_on', note=76, velocity=58, time=120))
track.append(Message('note_off', note=74, velocity=0, time=20))
track.append(Message('note_on', note=78, velocity=58, time=100))
track.append(Message('note_off', note=76, velocity=0, time=20))
track.append(Message('note_on', note=69, velocity=61, time=100))
track.append(Message('note_off', note=78, velocity=0, time=20))
track.append(Message('note_off', note=69, velocity=0, time=100))
track.append(Message('note_on', note=78, velocity=57, time=0))
track.append(Message('note_on', note=76, velocity=57, time=120))
track.append(Message('note_off', note=78, velocity=0, time=20))
track.append(Message('note_on', note=74, velocity=57, time=100))
track.append(Message('note_off', note=76, velocity=0, time=20))
track.append(Message('note_on', note=79, velocity=60, time=100))
track.append(Message('note_off', note=74, velocity=0, time=20))


mid.save('prompt.mid') # MidiFileを保存
